# 目标
爬取苦瓜书盘的数据，并下载图书文件。
支持单线程与多线程爬虫

![](http://881023.top/image/爬书战果.png)

In [2]:
import os
from requests_html import HTMLSession
import urllib
import urllib.request
import requests
# https
import ssl
ssl._create_default_https_context=ssl._create_unverified_context
# 线程池
from concurrent.futures import ThreadPoolExecutor,wait,as_completed

In [3]:
session = HTMLSession()
# 创建一个ua，绕过初级反爬
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0'
headers = {'User-Agent': ua}
# 创建一个最大数量为3的线程池
executor = ThreadPoolExecutor(max_workers=3)
# 跳过前几页
firstPage = 66

In [4]:
# 获取页面最后一页的地址,str
def get_last_page_url():
    # 返回一个 response 对象
    response = session.get('https://kgbook.com/list/index.html', headers=headers)
    pages = response.html.find('div.pagenavi', first=True)
    a_list = pages.find('a')
    urlObject = a_list[len(a_list) - 1]
    u = urlObject.absolute_links
    ustr = ''.join(u)
    return ustr;

# 获取页的数量
def get_total_pages(ustr):
    back = ustr.split('index_')[1]
    num = back.split('.')[0]
    print("一共%s页" % num)
    return int(num);
    
# 创建页地址数组
def get_total_page_urls(num):
    urls = []
    for index in range(num):
        if index <= 0:
            urls.append('https://kgbook.com/list/index.html')
        else:
            urls.append('https://kgbook.com/list/index_'+ str(index+1) +'.html')
    return urls
    
# 获取当前页面数据
def get_books_in_page(pageUrl, num):
    print('开始分析page', str(num+1), '>>>>>>>')
    s = session.get(pageUrl, headers=headers)
    books = s.html.find('div.channel-item a')
    #a_list = books.find('a')
    return books
    
# 根据一个bookdetailUrl下载书籍
def download_bookfile(pageNum, bookDetailUrl, bookname):
    print('开始解析下载', bookname, '当前页数', str(pageNum+1), '->->->')
    s = session.get(bookDetailUrl, headers=headers)
    btns = s.html.find('a.button')
    for btn in btns:
        bookurl = ''.join(btn.absolute_links)
        # 如果是下载链接
        if 'GetDown' in bookurl:
            realUrl, extendName = get_real_url(bookurl)
            bkname = bookname + '.' + extendName
            # 判断是不是已经存在了这本书
            if is_exist_book(bkname):
                print('\033[1;31m %s 已存在，跳过下载。 \033[0m' % bkname)
                continue
            downloadfile(realUrl, bkname)
    print(bookname, '下载完毕')
    print('-----------------------------')

# 下载文件
def downloadfile(url, filename):
    print('开始下载文件')
    try:
        # r = requests.get(url)
        path = "books/"+filename
        r = requests.get(url, headers=headers)
        with open(path,"wb") as f:
            f.write(r.content)
        f.close()
    except Exception:
        print('\033[1;31m 下载失败 \033[0m')
    else:
        print("\033[1;32m 下载", filename, "完成\033[0m")
    
# 获取重定向真实url
def get_real_url(url):
    print('开始获取真实下载地址')
    # http_headers = { 'Accept': '*/*','Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'}
    rs = requests.get(url,headers=headers)
    print(rs.url)
    # 获取文件的拓展名
    words = rs.url.split('.')
    extendName = words[len(words) - 1]
    print('这是一个', extendName, '文件')
    return rs.url, extendName

# 目录下是否存在这本书
def is_exist_book(bookname):
    if os.path.exists('books/'+bookname):
        return True
    else:
        return False


## 单线程进行书籍爬取

In [ ]:
# 全部最新更新书籍
url = get_last_page_url()
num = get_total_pages(url)
urls = get_total_page_urls(num)
# 遍历page
for index in range(len(urls)):
    #调整起始页
    if index < firstPage:
        print('跳过第%s页' % str(index+1))
        continue
    onepage = urls[index]
    bookset = get_books_in_page(onepage, index)
    # 遍历单页书籍
    for book in bookset:
        # 属性集合
        attrs = book.attrs
        # 如果属性里面没有title字段，说明不是一个书籍详情的链接
        if not 'title' in attrs.keys():
            continue
        title = book.text
        bookDetailUrl = ''.join(book.absolute_links)
        download_bookfile(index, bookDetailUrl, title)

## 多线程进行数据爬取

In [5]:
# 全部最新更新书籍
url = get_last_page_url()
num = get_total_pages(url)
urls = get_total_page_urls(num)
# 遍历page
for index in range(len(urls)):
    #调整起始页
    if index < firstPage:
        print('跳过第%s页' % str(index+1))
        continue
    onepage = urls[index]
    bookset = get_books_in_page(onepage, index)
    futures = []
    # 遍历单页书籍
    for book in bookset:
        # 属性集合
        attrs = book.attrs
        # 如果属性里面没有title字段，说明不是一个书籍详情的链接
        if not 'title' in attrs.keys():
            continue
        title = book.text
        bookDetailUrl = ''.join(book.absolute_links)
        # 把任务加入线程池队列
        print('把%s加入任务队列' % title)
        future = executor.submit(download_bookfile,index,bookDetailUrl,title)


一共81页
跳过第1页
跳过第2页
跳过第3页
跳过第4页
跳过第5页
跳过第6页
跳过第7页
跳过第8页
跳过第9页
跳过第10页
跳过第11页
跳过第12页
跳过第13页
跳过第14页
跳过第15页
跳过第16页
跳过第17页
跳过第18页
跳过第19页
跳过第20页
跳过第21页
跳过第22页
跳过第23页
跳过第24页
跳过第25页
跳过第26页
跳过第27页
跳过第28页
跳过第29页
跳过第30页
跳过第31页
跳过第32页
跳过第33页
跳过第34页
跳过第35页
跳过第36页
跳过第37页
跳过第38页
跳过第39页
跳过第40页
跳过第41页
跳过第42页
跳过第43页
跳过第44页
跳过第45页
跳过第46页
跳过第47页
跳过第48页
跳过第49页
跳过第50页
跳过第51页
跳过第52页
跳过第53页
跳过第54页
跳过第55页
跳过第56页
跳过第57页
跳过第58页
跳过第59页
跳过第60页
跳过第61页
跳过第62页
跳过第63页
跳过第64页
跳过第65页
跳过第66页
开始分析page 67 >>>>>>>
把肖申克救赎加入任务队列
开始解析下载 肖申克救赎 当前页数 67 ->->->
把湖加入任务队列
开始解析下载 湖 当前页数 67 ->->->
把阴阳师加入任务队列
开始解析下载 阴阳师 当前页数 67 ->->->
把风风雨雨一百年加入任务队列
把小窗幽记加入任务队列
把尘世挽歌加入任务队列
把关妖精的瓶子加入任务队列
把尼尔·盖曼中短篇科幻作品集加入任务队列
把罗杰疑案加入任务队列
把噬魂影加入任务队列
把1Q84加入任务队列
把国境以南太阳以西加入任务队列
把寻羊冒险记加入任务队列
把盗墓笔记1--4部加入任务队列
把Structure and Interpretation of Computer Programs加入任务队列
开始分析page 68 >>>>>>>
开始获取真实下载地址
开始获取真实下载地址
开始获取真实下载地址
把三重门加入任务队列
把人性的弱点加入任务队列
把职场谎言系列加入任务队列
把十万字的装修经验加入任务队列
把最后一名女知青加入任务队列
把狼图腾加入任务队列
把约翰克利斯多夫加入任务队列
把南禅七日加入任务队列
把西方哲学史加入任务队列
把哲学问题加入任务队列
把短线交易秘诀加入任务队列

https://kgbook.com/d/file/201011/2e115b7da4e18120fc046c0479338acb.pdf
这是一个 pdf 文件
 狼图腾.pdf 已存在，跳过下载。 
狼图腾 下载完毕
-----------------------------
开始解析下载 西方哲学史 当前页数 68 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/29ab1bed58d3ea101dd6cd8e696d7d98.pdf
这是一个 pdf 文件
 南禅七日.pdf 已存在，跳过下载。 
南禅七日 下载完毕
-----------------------------
开始解析下载 哲学问题 当前页数 68 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/99240a802bc55b0c6522e3add83ce1d0.pdf
这是一个 pdf 文件
 约翰克利斯多夫.pdf 已存在，跳过下载。 
约翰克利斯多夫 下载完毕
-----------------------------
开始解析下载 短线交易秘诀 当前页数 68 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/be9bcf26346f60344395274c04d1e4b1.pdf
这是一个 pdf 文件
 哲学问题.pdf 已存在，跳过下载。 
哲学问题 下载完毕
-----------------------------
开始解析下载 职场政治—华尔街人力资源案例 当前页数 68 ->->->
https://kgbook.com/d/file/201011/4d6a7387ac4c851ff62218e5b3d009d9.pdf
这是一个 pdf 文件
 西方哲学史.pdf 已存在，跳过下载。 
西方哲学史 下载完毕
-----------------------------
开始解析下载 当彩色的声音尝起来是甜的 当前页数 68 ->->->
开始获取真实下载地址
开始获取真实下载地址
https://kgbook.com/d/file/201011/fa7cf56a459e80c671c4dcfe744d5692.pd

https://kgbook.com/d/file/201011/c375e8984d8cf7f8d134dfd3fb4d4801.pdf
这是一个 pdf 文件
 小姨多鹤.pdf 已存在，跳过下载。 
小姨多鹤 下载完毕
-----------------------------
开始解析下载 中国哲学简史 当前页数 71 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/71ab79c6a4f17679bc2f0c5339efb641.pdf
这是一个 pdf 文件
 明末农民战争史.pdf 已存在，跳过下载。 
明末农民战争史 下载完毕
-----------------------------
开始解析下载 男人来自火星，女人来自金星 当前页数 71 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/704341fdaedf5329a9048446bbf55c4c.pdf
这是一个 pdf 文件
 中国哲学简史.pdf 已存在，跳过下载。 
中国哲学简史 下载完毕
-----------------------------
开始解析下载 犯罪心理名言警句集锦 当前页数 71 ->->->
https://kgbook.com/d/file/201011/799507e42779a764e6d561755194a1eb.pdf
这是一个 pdf 文件
 袁氏当国.pdf 已存在，跳过下载。 
袁氏当国 下载完毕
-----------------------------
开始解析下载 亡人相术·暗瞳 当前页数 71 ->->->
开始获取真实下载地址
开始获取真实下载地址
https://kgbook.com/d/file/201011/aacbd768beddd98510a0a98268f743bb.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201011/adee84ca151999990ca7d1b1f263ebc3.pdf
这是一个 pdf 文件
开始下载文件
 下载 男人来自火星，女人来自金星.pdf 完成
男人来自火星，女人来自金星 下载完毕
------------------------

开始获取真实下载地址
https://kgbook.com/d/file/201011/e9ef69c36e05aff995a1f7661c9ff888.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201011/a442797d9abdfd4bb801d5fceafe22d0.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201011/61185caf372c88d1549323108e97ae9d.pdf
这是一个 pdf 文件
开始下载文件
 下载 北方夕鹤2／3杀人事件.pdf 完成
北方夕鹤2／3杀人事件 下载完毕
-----------------------------
开始解析下载 蛤蟆的油 当前页数 73 ->->->
开始获取真实下载地址
 下载 哲学 科学 常识.pdf 完成
哲学 科学 常识 下载完毕
-----------------------------
开始解析下载 麦田里的守望者 当前页数 73 ->->->
开始获取真实下载地址
 下载 万圣节前夜的谋杀案.pdf 完成
万圣节前夜的谋杀案 下载完毕
-----------------------------
开始解析下载 海子诗全编 当前页数 74 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/f2a90b45d36a8c3d8f251048d6a81416.pdf
这是一个 pdf 文件
开始下载文件
 下载 蛤蟆的油.pdf 完成
蛤蟆的油 下载完毕
-----------------------------
开始解析下载 时间旅行者的妻子 当前页数 74 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/ca7c3f2068df7f0eae899874435cd1e6.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201011/2567e119e07798ffae9c3d58a57fa858.pdf
这是一个 pdf 文件
开始下载文件
 下载 麦田里的守望者.pdf 完成
麦田里的守望者 下载完毕
-------

 下载 呼兰河传.mobi 完成
开始获取真实下载地址
https://kgbook.com/d/file/201011/bdcccba517275339480d71ef25dc13bf.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201708/%E5%91%BC%E5%85%B0%E6%B2%B3%E4%BC%A0.epub
这是一个 epub 文件
开始下载文件
 下载 呼兰河传.epub 完成
呼兰河传 下载完毕
-----------------------------
开始解析下载 圈子圈套 当前页数 76 ->->->
开始获取真实下载地址
 下载 微软的梦工场.pdf 完成
微软的梦工场 下载完毕
-----------------------------
开始解析下载 做单 当前页数 76 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/b7915835b23b9c9ffcec8a2a470f0649.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201011/2b5b43d0fc2eb920bf8d81d68aa1e605.pdf
这是一个 pdf 文件
开始下载文件
 下载 科学史及其与哲学和宗教的关系.pdf 完成
科学史及其与哲学和宗教的关系 下载完毕
-----------------------------
开始解析下载 浮沉第2部 当前页数 76 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201011/3758644a54cc1c86300abac7c46105b1.pdf
这是一个 pdf 文件
开始下载文件
 下载 做单.pdf 完成
做单 下载完毕
-----------------------------
开始解析下载 浮沉 当前页数 76 ->->->
开始获取真实下载地址
 下载 浮沉第2部.pdf 完成
浮沉第2部 下载完毕
-----------------------------
开始解析下载 我们台湾这些年 当前页数 76 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/20

开始获取真实下载地址
 下载 银河英雄传说5-风云篇.pdf 完成
银河英雄传说5-风云篇 下载完毕
-----------------------------
开始解析下载 银河英雄传说2-野望篇 当前页数 78 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201009/c3ac3ea946f64cdaf732d7fc6fac3ad1.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201009/d2d8f469c79e625170cb58ff8e9a7902.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201009/e657db9a90620df3b16a698ec3efa389.pdf
这是一个 pdf 文件
开始下载文件
 下载 银河英雄传说4-策谋篇.pdf 完成
银河英雄传说4-策谋篇 下载完毕
-----------------------------
开始解析下载 银河英雄传说1-黎明篇 当前页数 78 ->->->
开始获取真实下载地址
 下载 银河英雄传说3-雌伏篇.pdf 完成
银河英雄传说3-雌伏篇 下载完毕
-----------------------------
开始解析下载 安徒生童话 当前页数 79 ->->->
开始获取真实下载地址
 下载 银河英雄传说2-野望篇.pdf 完成
银河英雄传说2-野望篇 下载完毕
-----------------------------
开始解析下载 巴菲特的真实故事 当前页数 79 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201009/fee152584086fc90dd4add0d19ff3c69.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201007/b61b001638deee518512cadd00fa8e84.pdf
这是一个 pdf 文件
开始下载文件
 下载 银河英雄传说1-黎明篇.pdf 完成
银河英雄传说1-黎明篇 下载完毕
-----------------------------
开始解析下载 漫长的战斗-美国人眼中的朝鲜战争

In [ ]:
executor.shutdown()